In [4]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS")
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS.get_dataframe()

# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

In [5]:
df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df

657500


In [18]:
# find all customer_names that have other customer names that start with this customer name

idx = 0
unique_customer_names = df['CUSTOMER'].unique()
full_set_n = len(unique_customer_names)

list_customers_ = []
list_potential_matches_ = []

report_every_n = 1000
save_every_n = 10000

for n in unique_customer_names:
    idx+=1
    df_f = df[(df['CUSTOMER'].str.startswith(n, na=False))&(df['CUSTOMER']!=n)]
    match_list = df_f['CUSTOMER'].unique()
    if len(match_list)>0:
        list_customers_.append(n)
        list_potential_matches_.append(match_list)

    if (idx % report_every_n == 0):
        print(idx, "iterations", len(list_customers_), "with potential matches", full_set_n-idx, "remaining", round(idx/full_set_n,2), "% complete")
        
    if (idx % save_every_n == 0):
        print('SAVING DATAFRAME')        
        df_candidates = pd.DataFrame(list_customers_)
        df_candidates.columns = ['CUSTOMER']
        df_candidates['POTENTIAL_MATCHES'] = list_potential_matches_

        ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df = df_candidates

        # Write recipe outputs
        ACCOUNTS_WITH_POTENTIAL_DUPLICATES = dataiku.Dataset("ACCOUNTS_WITH_POTENTIAL_DUPLICATES")
        ACCOUNTS_WITH_POTENTIAL_DUPLICATES.write_with_schema(ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df)
        print(len(ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df), "written")
        print("continuing on with checks....")
        
print(len(list_customers_))

100 iterations 6 with potential matches 657400 remaining 0.0 % complete
200 iterations 13 with potential matches 657300 remaining 0.0 % complete
300 iterations 16 with potential matches 657200 remaining 0.0 % complete
400 iterations 20 with potential matches 657100 remaining 0.0 % complete
500 iterations 23 with potential matches 657000 remaining 0.0 % complete
600 iterations 27 with potential matches 656900 remaining 0.0 % complete
700 iterations 31 with potential matches 656800 remaining 0.0 % complete
800 iterations 37 with potential matches 656700 remaining 0.0 % complete
900 iterations 42 with potential matches 656600 remaining 0.0 % complete
1000 iterations 48 with potential matches 656500 remaining 0.0 % complete
SAVING DATAFRAME
48 rows successfully written (19NaGQli90)
48 written
continuing on with checks....
1100 iterations 51 with potential matches 656400 remaining 0.0 % complete
1200 iterations 58 with potential matches 656300 remaining 0.0 % complete


KeyboardInterrupt: 